# Cassandra
environment setup

- switched to conda mosdef cassandra

```
conda create -n cassandra -y -c conda-forge -c mosdef -c omnia 'python=3.7' mbuild foyer openbabel py3dmol fresnel freud jupyterlab mosdef_cassandra

conda activate cassandra
```

In [3]:
%%bash

# Delete output of previous run
rm -rf *.in.* species* *.out* *.inp*

In [ ]:
# Don't run this when testing because it's a bad idea, but also there are way too many warnings :/
import warnings

warnings.simplefilter("ignore")

In [84]:
import mosdef_cassandra as mc
import foyer
import mbuild as mb
import mosdef_cassandra.examples as ex

In [2]:
# If this fails, you need to add Cassandra to your path
# export PATH=/Users/$(whoami)/Projects/Cassandra_V1.2/bin:${PATH}
mc.utils.detect_cassandra_binaries();

Using the following executables for Cassandra:
Python: /Users/jenny/miniconda3/envs/cassandra/bin/python
library_setup: /Users/jenny/miniconda3/envs/cassandra/bin/library_setup.py
Cassandra: /Users/jenny/miniconda3/envs/cassandra/bin/cassandra.exe


In [3]:
lattice = ex.structures.carbon_lattice()
#lattice.visualize(color_scheme={"_CH4": "grey"}).show()

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/lattice.py:584: UserWarning: Periodicity of non-rectangular lattices are not valid with default boxes. Only rectangular lattices are valid at this time.
  warn('Periodicity of non-rectangular lattices are not valid with '


In [4]:
methane = mb.load("C", smiles=True)
#methane.visualize().show()

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule C	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


In [5]:
# Load forcefields
trappe = foyer.forcefields.load_TRAPPE_UA()
oplsaa = foyer.forcefields.load_OPLSAA()

# Use foyer to apply forcefields
typed_lattice = trappe.apply(lattice)
typed_methane = oplsaa.apply(methane)

# Create box and species list
box_list = [lattice]
species_list = [typed_lattice, typed_methane]

# Since we have an occupied box we need to specify
# the number of each species present in the intial config
mols_in_boxes = [[1, 0]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moves = mc.Moves("gcmc", species_list)

custom_args = {
    "chemical_potentials": ["none", -30.0],
    "rcut_min": 0.5,
    "vdw_cutoff": 14.0,
    "charge_cutoff": 14.0,
    "coord_freq": 100,
    "prop_freq": 10,
}

mc.run(system, moves, "equilibration", 10000, 300.0, **custom_args)

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:457: UserWarning: Non-atomistic element type detected. Creating custom element for _CH4
  'Creating custom element for {}'.format(element))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:457: UserWarning: Non-atomistic element type detected. Creating custom element for _CH3
  'Creating custom element for {}'.format(element))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:457: UserWarning: Non-atomistic element type detected. Creating custom element for _CH2
  'Creating custom element for {}'.format(element))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:457: UserWarning: Non-atomistic element type detected. Creating custom element for _HC
  'Creating custom element for {}'.format(element))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:433: U

Treating <Structure 320 atoms; 1 residues; 0 bonds; PBC (orthogonal); NOT parametrized> as a non-insertable rigid species since it has no bonds
Using the following executables for Cassandra:
Python: /Users/jenny/miniconda3/envs/cassandra/bin/python
library_setup: /Users/jenny/miniconda3/envs/cassandra/bin/library_setup.py
Cassandra: /Users/jenny/miniconda3/envs/cassandra/bin/cassandra.exe
Generating fragment libraries...


/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:178: UserWarning: No bonds found. Cassandra will interpet this as a rigid species
  warnings.warn("No bonds found. Cassandra will interpet "
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:564: UserWarning: More than two atoms present but no fragments identified.
  warnings.warn('More than two atoms present but '
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:303: UserWarning: Warning, type name opls_138 will be shortened to six characters as ls_138. Please confirm your final MCF.
  "MCF.".format(itype,itype[-6:]))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:303: UserWarning: Warning, type name opls_140 will be shortened to six characters as ls_140. Please confirm your final MCF.
  "MCF.".format(itype,itype[-6:]))
/Users/jenny/miniconda3

Running Cassandra...


In [9]:
%%bash

# move the output
mkdir adsorption
mv *.in.* species* *.out* *.inp* adsorption

In [2]:
import ex_render_cassandra as erc

In [3]:
xyzfile = "adsorption/gcmc.out.xyz"
boxfile = "adsorption/gcmc.out.H"

traj = erc.make_traj(xyzfile, boxfile)

In [5]:
erc.display_movie(erc.render_sphere_frame, traj, "traj.gif")

The above functions saves a gif of the rendered trajectory-- this is the gif displayed in markdown

![](traj.gif)

It can also be displayed inline as an IPython widget if the gif name is omitted

`erc.display_movie(erc.render_sphere_frame, traj)`

In [6]:
erc.display_movie(erc.render_sphere_frame, traj)

Notes
- gcmc.out.H - box
- gcmc.out.xyz - particle coordinates
- species#.mcf - connectivity
- Cassandra treats no bonds as fixed - check the inp file (max distances for the frozen species should be 0)
- this file `species2/frag1/frag1.dat` contains the fragment configurations 
- --changing the temp (300) to natoms (5 for methane) makes it viewable by vmd (watch the fragment wiggle around)
- chk - restart (mc.restart)
- log - energies, etc
- properties - thermo properties
- cassandra uses fixed bonds -- make sure ring systems have correct bonds!